In [56]:
# COl759
import numpy as np

In [57]:
#for char to int where a=0...z=25
def charToInt(s):
    value=ord(s)-ord('a')
    return value

#for int to char where 0=a...25=z
def intTochar(s):
    return chr(s%26 +ord('a'))

In [58]:
def IdCoincidence(charOrdList):
    freq ={}
    N = len(charOrdList)
    for i in charOrdList:
        if i in freq:
            freq[i]+=1
        else:
            freq[i]=1
    num = 0
    for i in freq:
        num += freq[i]*(freq[i]-1)
    ic = num/(N*(N-1))
    return ic

In [59]:
def ngram():
    bigram = ["th", "he", "in", "er", "an", "re", "nd", "at", "on", "nt"]
    trigram = ["the", "and", "ing", "ent", "ion", "tio", "for", "nde", "has", "nce"]
    return bigram, trigram



In [60]:
def frequency(cipherTextPairs):
    freqDict ={}
    for i in cipherTextPairs:
        if i in freqDict:
            freqDict[i]+=1
        else:
            freqDict[i]=1
    return freqDict

In [61]:
def inputKeySize():
    print("Enter the keySize \n")
    print("cipher text is taken automatically from ./InputFiles/cipherText.txt")
    keySize = int(input())
    cipher = open("./InputFiles/cipherText.txt",'r')
    cipherTextList = list(cipher.read().replace("\n","").replace(" ",""))
    return keySize, cipherTextList


def ngramGenerator(keySize,cipherTextList):
    cipherTextPairs = np.reshape(np.array(cipherTextList),(-1,keySize))
    ngram = []
    for i in cipherTextPairs:
        ngram.append("".join(i))
    return ngram
    

In [62]:
def topTenFreq():
    (keySize, cipherTextList) = inputKeySize()
#     print(cipherTextList)
    ngram = ngramGenerator(keySize, cipherTextList)
    freqDict = frequency(ngram)
    sortedfreq = sorted(freqDict , key = freqDict.get)[::-1]
    return sortedfreq[:10] , keySize   


In [63]:
# Ax+My = 1 = gcd(A,M) = gcd(M,A%M)
x=1
y=0
cf=0  #highest commmon factor
def gcd(A,M):
    global x
    global y
    global cf
    if M==0:
        x=1
        y=0
        cf=A
    else:
        gcd(M,A%M)
        temp=x
        x=y
        y=temp-(A//M)*y
#         print(x,y,cf)

In [64]:
##takes input key and cpherText in" int np.array" format
def decryption(K,C):
    PT=[np.dot(K,CT) for CT in C]
    return list(map(intTochar,list(np.reshape(PT,(1,-1)).flat)))

In [65]:
def invKey(key,keySize):
    
    det = np.round(np.linalg.det(key))

    gcd(det,26)
    global cf
    global x
    if(cf!=1):
        return np.zeros((keySize,keySize))

    invKey = np.linalg.inv(key)

    adj = np.round(invKey*det)*(x%26) #cf is the modular inverse of det w.r.t 26

    keymodInv = np.mod(adj.astype(int),26)
    return keymodInv

In [66]:
def cipherTextinput():
    with open("./InputFiles/cipherText.txt","r") as chipherTextFile:
        cipherText = chipherTextFile.read().replace("\n","").replace(" ","")
        return list(map(charToInt,list(cipherText)))

In [67]:
def decrypt(keymodInv,keySize):
    cipherTextList = cipherTextinput()
    cipherTextMatrix = np.reshape(cipherTextList,(-1,keySize))
    plainTextList=decryption(keymodInv,cipherTextMatrix)
    s = "".join(plainTextList)
    return s

In [71]:
def cryptAnalysis():
    opFile = open("analysis","w+")
    (topTenCipherList, keySize) = topTenFreq()
    (Eng_bigram, Eng_trigram) = ngram()
    if keySize ==2:
        Eng_topTen = Eng_bigram
    else:
        Eng_topTen = Eng_trigram
        
#     print(Eng_topTen)
#     print(topTenCipherList)
    
    count=0
    for i in range(10):
        for j in range(i+1,10):
            for k in range(10):
                for l in range(k+1,10):
                    count+=1
                    eng1 = np.reshape(np.array(list(map(charToInt,list(Eng_topTen[i])))),(keySize,-1))
                    eng2 = np.reshape(np.array(list(map(charToInt,list(Eng_topTen[j])))),(keySize,-1))
                    ciph1 = np.reshape(np.array(list(map(charToInt,list(topTenCipherList[k])))),(keySize,-1))
                    ciph2 = np.reshape(np.array(list(map(charToInt,list(topTenCipherList[l])))),(keySize,-1))
                    
                    Cmatrix1 = np.append(ciph1,ciph2,axis = 1)
                    Cmatrix2 = np.append(ciph2,ciph1,axis = 1)
                    Pmatrix1 = np.append(eng1,eng2,axis = 1)
                    Pmatrix2 = np.append(eng2,eng1,axis = 1)
                    
                    
                    Pmatrix1_inv = invKey(Pmatrix1, keySize)
                    if not np.array_equal(Pmatrix1_inv , np.zeros((keySize,keySize)) ): 
                        key1 =np.mod( np.dot(Cmatrix1,Pmatrix1_inv),26)
                        key1Inv = invKey(key1,keySize)
                        key1 =list(key1.flat)
                        PT = decrypt(key1Inv.astype(int),keySize)
                        ic = IdCoincidence(list(PT))
                        if (ic > 0.06) and (ic < 0.07):
                            opFile.write(str(key1))
                            opFile.write("\n")
                            opFile.write(PT)
                            opFile.write("\n")
                        
                        key2 = np.mod(np.dot(Cmatrix2,Pmatrix1_inv), 26)
                        key2Inv = invKey(key2,keySize)
                        key2 =list(key2.flat)
                        PT = decrypt(key2Inv.astype(int),keySize)
                        ic = IdCoincidence(list(PT))
                        if (ic > 0.06) and (ic < 0.07):
                            opFile.write(str(key2))
                            opFile.write("\n")
                            opFile.write(PT)
                            opFile.write("\n")
                        
                    Pmatrix2_inv = invKey(Pmatrix2, keySize) 
                    if not np.array_equal(Pmatrix2_inv , np.zeros((keySize,keySize)) ): 
                        key3 = np.mod(np.dot(Cmatrix1,Pmatrix2_inv), 26)
                        key3Inv = invKey(key3,keySize)
                        key3 =list(key3.flat)
                        PT = decrypt(key3Inv.astype(int),keySize)
                        ic = IdCoincidence(list(PT))
                        if (ic > 0.06) and (ic < 0.07):
                            opFile.write(str(key3))
                            opFile.write("\n")
                            opFile.write(PT)
                            opFile.write("\n")
                        
                        key4 = np.mod(np.dot(Cmatrix2,Pmatrix2_inv), 26)
                        key4Inv = invKey(key4,keySize)
                        key4 =list(key4.flat)
                        PT = decrypt(key4Inv.astype(int),keySize)
                        ic = IdCoincidence(list(PT))
                        if (ic > 0.06) and (ic < 0.07):
                            opFile.write(str(key4))
                            opFile.write("\n")
                            opFile.write(PT)
                            opFile.write("\n")
                    
                    
                    
                    
                    
                    
    opFile.close()
 
            
            

In [72]:
cryptAnalysis()

Enter the keySize 

cipher text is taken automatically from ./InputFiles/cipherText.txt
2
